<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=fc63dd71490a35e42d31e243b44a1db2d02333b9e58e24c794ffdc2950239a9d
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-11 11:18:51
-------------------
qualified stocks: 86
with latest results: 24
still star stocks: 15
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  99.50 K
Current:  1.42 C
-------------------
Today PnL: -67.52 K (-0.47%)
Current PnL: -22.82 L (-14.95%)
CY Booked + Current PnL: -9.63 L (-6.31%)
-------------------
Total profit:  1.47 L
Total loss:  -24.28 L
-------------------
Total Booked + Current PnL: 17.35 L (13.81%)
Total Booked PnL: 40.17 L (31.98%)
Curr Year Booked PnL: 13.19 L (9.27%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.33 C
Est FTT PnL: 90.28 L (63.44%)
Deployed:  1.26 C
Current:  1.42 C
CAGR/XIRR %: 7.25%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TTKPRESTIG,770.00,101.07,50.0,M-SC,2.34,86611.0,-14166.0,14256.0,0.08,-14.06,16.46,0.09,245.0,-0.99,0.61,13.79,OX40N,NTT,DURABLES
2,ABBOTINDIA,35195.00,-10.25,47.0,X-MC,3.15,88665.0,-2013.0,16917.0,0.17,-2.22,19.08,16.44,101.0,-0.12,0.63,17.51,X40,ATH,PHARMA
42,ITC,452.00,-38.77,44.0,X-LC,0.84,198241.0,-1897.0,22778.0,-0.04,-0.95,11.49,10.44,4.0,-0.08,1.40,4.56,X40,NTT,FMCG
34,ICICIGI,2252.93,-14.31,53.0,X-MC,6.58,185265.0,10945.0,24251.0,-1.64,6.28,13.09,20.19,91.0,0.45,1.31,22.34,X40,ATH,INSURANCE
51,NESTLEIND,1377.00,-4.02,58.0,X-LC,8.21,295448.0,30022.0,26768.0,-0.02,11.31,9.06,21.40,11.0,1.12,2.09,18.71,XY25,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDUSINDBK,1800.0,283.05,71.0,L-MC,7.38,47014.0,-30192.0,55585.0,3.18,-39.11,118.23,32.89,258.0,-0.54,0.33,29.47,XR,NTT,BANKS
13,BERGEPAINT,680.0,-17.83,55.0,X-MC,1.09,221029.0,-6310.0,54373.0,1.90,-2.78,24.60,21.14,106.0,-0.12,1.56,24.58,XY24,NTT,PAINTS
38,INDIGOPNTS,1408.0,121.54,75.0,M-SC,5.37,145396.0,-29163.0,29196.0,1.49,-16.71,20.08,0.02,221.0,-1.00,1.03,26.47,OX40N,NTT,PAINTS
74,TATAMOTORS,1065.0,-53.17,24.0,X-LC,36.18,148993.0,-127530.0,237599.0,1.17,-46.12,159.47,39.81,54.0,-0.54,1.05,5.01,XY24,NTT,AUTO
79,UNITDSPR,1644.0,-6.52,61.0,X-LC,3.60,244997.0,10853.0,37779.0,1.06,4.64,15.42,20.77,86.0,0.29,1.73,11.87,X40N,NTT,BREWERIES


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
80,VAIBHAVGBL,521.00,64.19,53.0,H-SC,5.50,142128.0,-40647.0,151722.0,-3.89,-22.24,106.75,60.77,125.0,-0.27,1.01,28.00,XR,NTT,JEWELLERY
47,KPIGREEN,731.05,2.50,43.0,H-SC,9.39,122044.0,-3253.0,62181.0,-3.76,-2.60,50.95,47.03,141.0,-0.05,0.86,52.81,MH,ATH,POWER
60,ROUTE,2227.21,-59.29,21.0,H-SC,23.30,67135.0,-65931.0,157814.0,-2.59,-49.55,235.07,69.05,118.0,-0.42,0.48,0.00,SR,ATH,IT
84,WHIRLPOOL,2270.00,-44.45,39.0,M-SC,8.41,115230.0,432.0,93613.0,-2.54,0.38,81.24,81.92,223.0,0.00,0.82,36.41,XR,NTT,DURABLES
25,GILLETTE,11206.78,-18.99,16.0,X-MC,5.55,248760.0,-5136.0,87439.0,-2.17,-2.02,35.15,32.42,105.0,-0.06,1.76,12.77,X40,ATH,FMCG


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WHIRLPOOL,2270.0,-44.45,39.0,M-SC,8.41,115230.0,432.0,93613.0,-2.54,0.38,81.24,81.92,223.0,0.0,0.82,36.41,XR,NTT,DURABLES


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TTKPRESTIG,770.0,101.07,50.0,M-SC,2.34,86611.0,-14166.0,14256.0,0.08,-14.06,16.46,0.09,245.0,-0.99,0.61,13.79,OX40N,NTT,DURABLES
38,INDIGOPNTS,1408.0,121.54,75.0,M-SC,5.37,145396.0,-29163.0,29196.0,1.49,-16.71,20.08,0.02,221.0,-1.00,1.03,26.47,OX40N,NTT,PAINTS
46,KANSAINER,340.0,-68.16,40.0,H-SC,2.27,219456.0,-50211.0,86553.0,-0.81,-18.62,39.44,13.47,138.0,-0.58,1.55,11.59,XY24,NTT,PAINTS
17,CERA,9475.0,-25.06,40.0,H-SC,1.80,136700.0,-39203.0,81227.0,0.34,-22.29,59.42,23.89,149.0,-0.48,0.97,18.06,OX40N,NTT,CERAMICS
67,SIS,528.0,2035.23,49.0,H-SC,2.17,86164.0,-24868.0,48476.0,0.94,-22.40,56.26,21.26,156.0,-0.51,0.61,16.42,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAIL,228.00,55.57,65.0,M-MC,10.42,245385.0,20423.0,147231.0,-1.27,9.08,60.00,74.53,192.0,0.14,1.74,44.26,XY24,BTT,STEEL
32,HINDZINC,730.22,29.16,52.0,M-LC,9.15,210018.0,4942.0,107634.0,-0.37,2.41,51.25,54.89,52.0,0.05,1.49,27.09,X5K,ATH,METALS
84,WHIRLPOOL,2270.00,-44.45,39.0,M-SC,8.41,115230.0,432.0,93613.0,-2.54,0.38,81.24,81.92,223.0,0.00,0.82,36.41,XR,NTT,DURABLES
37,INDIAMART,4810.62,-53.89,49.0,H-SC,5.06,128848.0,5512.0,126116.0,-0.84,4.47,97.88,106.72,119.0,0.04,0.91,28.89,AR,ATH,MISC
62,SAMMAANCAP,326.00,-192.05,54.0,M-SC,35.16,159075.0,9855.0,134323.0,-1.39,6.60,84.44,96.62,208.0,0.07,1.13,72.15,XY25,NTT,FINANCE


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,HINDZINC,730.22,29.16,52.0,M-LC,9.15,210018.0,4942.0,107634.0,-0.37,2.41,51.25,54.89,52.0,0.05,1.49,27.09,X5K,ATH,METALS
37,INDIAMART,4810.62,-53.89,49.0,H-SC,5.06,128848.0,5512.0,126116.0,-0.84,4.47,97.88,106.72,119.0,0.04,0.91,28.89,AR,ATH,MISC
84,WHIRLPOOL,2270.00,-44.45,39.0,M-SC,8.41,115230.0,432.0,93613.0,-2.54,0.38,81.24,81.92,223.0,0.00,0.82,36.41,XR,NTT,DURABLES
85,WIPRO,420.00,-15.61,45.0,M-LC,5.71,148577.0,-2368.0,111819.0,-0.08,-1.57,75.26,72.51,53.0,-0.02,1.05,4.36,XR,NTT,IT
47,KPIGREEN,731.05,2.50,43.0,H-SC,9.39,122044.0,-3253.0,62181.0,-3.76,-2.60,50.95,47.03,141.0,-0.05,0.86,52.81,MH,ATH,POWER


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,GILLETTE,11206.78,-18.99,16.0,X-MC,5.55,248760.0,-5136.0,87439.0,-2.17,-2.02,35.15,32.42,105.0,-0.06,1.76,12.77,X40,ATH,FMCG
74,TATAMOTORS,1065.00,-53.17,24.0,X-LC,36.18,148993.0,-127530.0,237599.0,1.17,-46.12,159.47,39.81,54.0,-0.54,1.05,5.01,XY24,NTT,AUTO
45,JSWINFRA,342.00,-24.78,26.0,X-MC,4.45,188662.0,-11685.0,42185.0,0.25,-5.83,22.36,15.23,178.0,-0.28,1.34,24.54,X40N,NTT,REALTY
12,BATAINDIA,2096.00,-42.01,27.0,X-SC,8.50,86793.0,-41877.0,87175.0,-0.36,-32.55,100.44,35.20,219.0,-0.48,0.61,0.00,X40,NTT,FOOTWEAR
23,DMART,5391.45,-20.66,30.0,X-LC,6.77,202130.0,-8277.0,67451.0,0.59,-3.93,33.37,28.12,38.0,-0.12,1.43,18.76,X40N,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,PGHH,17907.65,-31.74,35.0,X-MC,0.65,197460.0,-3360.0,71165.0,-0.90,-1.67,36.04,33.76,80.0,-0.05,1.40,3.44,X40,ATH,FMCG
42,ITC,452.00,-38.77,44.0,X-LC,0.84,198241.0,-1897.0,22778.0,-0.04,-0.95,11.49,10.44,4.0,-0.08,1.40,4.56,X40,NTT,FMCG
66,SIEMENS,4671.50,-5.82,37.0,H-LC,1.62,151400.0,-34695.0,82180.0,-0.86,-18.64,54.28,25.51,15.0,-0.42,1.07,11.57,AR,ATH,ELECTRICAL
29,HAVELLS,2069.16,-9.44,35.0,X-MC,3.32,302400.0,-25591.0,132119.0,-1.21,-7.80,43.69,32.48,92.0,-0.19,2.14,2.73,X40,ATH,ELECTRICAL
79,UNITDSPR,1644.00,-6.52,61.0,X-LC,3.60,244997.0,10853.0,37779.0,1.06,4.64,15.42,20.77,86.0,0.29,1.73,11.87,X40N,NTT,BREWERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BATAINDIA,2096.00,-42.01,27.0,X-SC,8.50,86793.0,-41877.0,87175.0,-0.36,-32.55,100.44,35.20,219.0,-0.48,0.61,0.00,X40,NTT,FOOTWEAR
54,QUESS,986.00,-51.52,32.0,X-SC,37.26,48134.0,-16872.0,167800.0,-1.84,-25.95,348.61,232.18,198.0,-0.10,0.34,0.00,XY24,NTT,MISC
20,COLPAL,3726.84,-7.61,38.0,X-MC,7.15,215690.0,-47675.0,157001.0,-0.66,-18.10,72.79,41.51,84.0,-0.30,1.53,1.22,XY25,ATH,FMCG
1,ABB,7934.00,-42.98,32.0,H-LC,5.00,238320.0,-23299.0,142515.0,-0.54,-8.91,59.80,45.57,7.0,-0.16,1.69,1.41,AR,NTT,ELECTRICAL
52,PAGEIND,51769.93,-29.12,38.0,X-MC,7.64,160280.0,-3460.0,46802.0,-0.32,-2.11,29.20,26.47,82.0,-0.07,1.13,1.55,X40,ATH,APPARELS


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,986.00,-51.52,32.0,X-SC,37.26,48134.0,-16872.0,167800.0,-1.84,-25.95,348.61,232.18,198.0,-0.10,0.34,0.00,XY24,NTT,MISC
58,RELAXO,1176.00,-40.85,40.0,X-SC,4.04,80661.0,-64499.0,140423.0,0.26,-44.43,174.09,52.31,136.0,-0.46,0.57,7.97,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,-42.01,27.0,X-SC,8.50,86793.0,-41877.0,87175.0,-0.36,-32.55,100.44,35.20,219.0,-0.48,0.61,0.00,X40,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-10.25,47.0,X-MC,3.15,88665.0,-2013.0,16917.0,0.17,-2.22,19.08,16.44,101.0,-0.12,0.63,17.51,X40,ATH,PHARMA
33,HONAUT,58357.33,-23.99,37.0,X-SC,0.90,107055.0,-20883.0,68012.0,-0.92,-16.32,63.53,36.84,143.0,-0.31,0.76,9.60,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TCS,4389.97,-28.69,51.0,X-LC,9.38,284538.0,-61418.0,128127.0,0.06,-17.75,45.03,19.28,1.0,-0.48,2.01,5.19,X40,ATH,IT
40,INFY,2275.00,-16.73,62.0,X-LC,4.71,323824.0,10936.0,160746.0,0.45,3.50,49.64,54.87,3.0,0.07,2.29,12.07,X40,BTT,IT
42,ITC,452.00,-38.77,44.0,X-LC,0.84,198241.0,-1897.0,22778.0,-0.04,-0.95,11.49,10.44,4.0,-0.08,1.40,4.56,X40,NTT,FMCG
82,VBL,671.64,-17.93,49.0,X-LC,5.76,295681.0,-20161.0,132820.0,0.04,-6.38,44.92,35.67,5.0,-0.15,2.09,6.59,X40N,ATH,FMCG
1,ABB,7934.00,-42.98,32.0,H-LC,5.00,238320.0,-23299.0,142515.0,-0.54,-8.91,59.80,45.57,7.0,-0.16,1.69,1.41,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
19,COFFEEDAY,80.00,-47.69,41.0,L-SC,27.22,76378.0,-37171.0,77226.0,-2.12,-32.74,101.11,35.27,268.0,-0.48,0.54,86.94,XR,NTT,HOTELS
7,ASIANTILES,137.00,7438.89,62.0,L-SC,13.97,82726.0,-11084.0,87706.0,-1.61,-11.82,106.02,81.67,269.0,-0.13,0.59,60.20,XR,NTT,CERAMICS
50,MASFIN,398.61,-19.60,48.0,H-SC,7.93,91575.0,-6405.0,28004.0,0.08,-6.54,30.58,22.05,152.0,-0.23,0.65,33.11,XR,ATH,FINANCE
47,KPIGREEN,731.05,2.50,43.0,H-SC,9.39,122044.0,-3253.0,62181.0,-3.76,-2.60,50.95,47.03,141.0,-0.05,0.86,52.81,MH,ATH,POWER
80,VAIBHAVGBL,521.00,64.19,53.0,H-SC,5.50,142128.0,-40647.0,151722.0,-3.89,-22.24,106.75,60.77,125.0,-0.27,1.01,28.00,XR,NTT,JEWELLERY


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ASIANTILES,137.00,7438.89,62.0,L-SC,13.97,82726.0,-11084.0,87706.0,-1.61,-11.82,106.02,81.67,269.0,-0.13,0.59,60.20,XR,NTT,CERAMICS
80,VAIBHAVGBL,521.00,64.19,53.0,H-SC,5.50,142128.0,-40647.0,151722.0,-3.89,-22.24,106.75,60.77,125.0,-0.27,1.01,28.00,XR,NTT,JEWELLERY
6,ASIANPAINT,3465.66,-7.96,72.0,X-LC,6.15,232320.0,-19448.0,72646.0,-0.39,-7.72,31.27,21.13,27.0,-0.27,1.64,24.96,X40,ATH,PAINTS
38,INDIGOPNTS,1408.00,121.54,75.0,M-SC,5.37,145396.0,-29163.0,29196.0,1.49,-16.71,20.08,0.02,221.0,-1.00,1.03,26.47,OX40N,NTT,PAINTS
5,ANGELONE,3033.00,20.92,66.0,X-SC,6.64,214299.0,23293.0,34416.0,0.27,12.19,16.06,30.21,157.0,0.68,1.52,35.62,X40N,NTT,FINANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.81
1,25,44.77
2,50,76.12


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.53
LC    30.47
MC    26.00
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.76
X40      19.31
X40N     12.46
XY25     10.24
XR        9.70
AR        7.74
OX40N     6.63
X200      1.79
X5K       1.49
SR        1.02
MH        0.86
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    23.95
X-MC    21.61
X-LC    20.99
M-SC    12.55
X-SC     5.57
H-LC     4.55
M-LC     3.93
H-MC     2.32
M-MC     1.74
L-SC     1.46
L-LC     1.00
L-MC     0.33
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.01,-4.71,38.57
IT,12.61,-17.97,81.11
FINANCE,12.14,-10.40,60.45
MISC,7.72,-19.12,79.20
PAINTS,5.78,-12.85,29.67
ELECTRICAL,5.66,-13.07,53.16
INSURANCE,4.46,-0.12,35.41
AUTO,2.71,-49.71,112.19
BANKS,2.70,-39.80,130.91


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3323545.0,21
XR,1287846.0,13
AR,1194571.0,9
X40,959672.0,13
X40N,708703.0,9
OX40N,546952.0,9
XY25,488857.0,7
SR,277484.0,2
X5K,107634.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3449638.0,24
X-MC,1354445.0,15
M-SC,1271874.0,15
X-LC,1070250.0,12
X-SC,562163.0,6
H-LC,295291.0,3
M-LC,268325.0,3
H-MC,259435.0,2
L-SC,255169.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1197647.0      6
           AR         844558.0      5
           XR         812767.0      7
M-SC       XY24       742575.0      6
X-MC       XY24       571419.0      4
X-LC       X40        442424.0      5
X-MC       X40        430073.0      7
X-LC       XY24       312237.0      2
X-SC       X40N       307188.0      4
H-SC       SR         277484.0      2
           OX40N      255001.0      3
X-LC       X40N       238050.0      3
H-LC       AR         224695.0      2
X-MC       XY25       189488.0      2
H-MC       XY24       184636.0      1
X-SC       XY24       167800.0      1
L-SC       XR         164932.0      2
X-MC       X40N       163465.0      2
M-MC       XY24       147231.0      1
M-SC       XR         142743.0      2
           XY25       134323.0      1
           OX40N      126915.0      4
           AR         125318.0      2
M-LC       XR         111819.0      1
           X5K        107634.0      1
L-SC       OX40N       90237.0      1
X-SC       X40         87175.0      1
X-LC       XY25        77539.0      2
H-MC       OX40N       74799.0      1
H-LC       X200        70596.0      1
H-SC       MH          62181.0      1
L-MC       XR          55585.0      1
M-LC       XY25        48872.0      1
L-LC       XY25        38635.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 45.0 seconds
